<a href="https://colab.research.google.com/github/AguaClara/CEE4520/blob/master/Unit_Process_Designs/Entrance_tank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrance Tank

**Note:** Outdated executive summaries moved to 'Information and Notes' section.






# Design the Entrance Tank in Python

*The entrance tank will morph from option 2 (like Gracias) to option 3 (full length of the flocculator) to wider (to get even better removal of grit). Determine the number of hoppers depending on the L/W ratio.* 

**Onshape Drawing**
- Problem with drawing the hoppers when the number of hoppers is 1. Linear pattern does not accept 0 as a number of instances.

**Comments/questions on the design**
- chamber or pipe below the entrance tank to connect to drains
- are we sure that we want a grit chamber inside the plant, opposed to by the intake?
- if the et gets veryyy wide, what if we add a second influent pipe in the tank?
- should the code give the option for an even or odd number of channels? **wait to get feedback from APP**

**Things to do:**
- recalculate tRack depth in a way that is more useful for Onshape, include freeboard for trash rack
- determine how deep the influent orifices should be submerged below the surface. determine the orifice diameter and the diameter of the influent pipe
- influent pipe design (add obstacles to back end of the tank, what material?) **wait to get feedback from APP**
- add code for lfom port geometry if needed
- get feedback on algorithm for determining ET and Flocc dimensions
- sanity checks...everywhere! (check that drain pipe is 2 or 3 inches)
- rename variables to match naming guidelines

**Materials:**
[Trash Rack Mesh](https://www.mcmaster.com/stainless-steel-mesh/stainless-steel-wire-cloth-9/) , 
[Trash Rack Tracks](https://www.mcmaster.com/standard-aluminum-u-channels/)





In [7]:
#### Official Code ####

"""The entrance tank of an AguaClara water treatment plant"""

!pip install aguaclara
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import numpy as np
from aguaclara.design.component import Component
from aguaclara.design.pipeline import Pipe


class EntranceTank(Component):
  """Design an AguaClara plant's entrance tank. 

     An entrance tank's design relies on the LFOM's and the floccualtor's design in
     the same plant, but assumed/default values may be used to design an entrance
     tank by itself. To design these components in tandem, use :class:`aguaclara.design.ent_floc.EntTankFloc`.

     Design Inputs:
        - ``q (float * u.L / u.s)``: Flow rate (recommended, defaults to 60 L/s)
        - ``temp (float * u.degC)``: Water temperature (recommended, defaults to 20°C)
        - ``diam_p (float * u.mm)``: The smallest particle that needs to be captured by the entrance tank grit chamber (optional, defaults to 2.5 mm)
        - ``lfom_od (float * u.m)``: The LFOM's outer diameter (recommended, defaults to 10 inch")
        - ``lfom_hl (float * u.m)``: The head loss through the lfom (recommended, defaults to 30 cm)
        - ``flocc_gt (float)``: Flocculator collision potential (optional, defaults to 37000)
        - ``flocc_hl (float * u.cm)``: Flocculator head loss (optional, defaults to 40cm)
        - ``flocc_widthMax (float * u.m)``: The maximum width of a flocculator channel based on the polycarbonate sheets used (optional, defaults to 1.08 m)
        - ``flocc_widthMin (float * u.m)``: The minimum width of a flocculator based on average human width (optional, defaults to 0.45 m) 
        - ``flocc_depth (float)``: The depth of the flocculator (optional, defaults to 2.17 m)
        - ``dividing_wall_thickness (float * u.m)``: Thickness of dividing walls between each channel (optional, defaults to 15 cm)
        - ``sed_length (float * u.m)``: the length of a sedimentation tank plus the width of the influent and effluent sedimentation channels (recommended, defaults to 7.1 m)
        - ``fab_s(float * u.cm)``: The space needed for a person to remove the drain pipe (optional, defaults to 5 cm)
        - ``hopper_LWratio (float)``: The length:width ratio of a hopper. Used to manipulate the tradeoff between hopper depth and number of hoppers (optional, defaults to 1)
        - ``influentHopper_LWratio (float)``: The length:width ratio of a hopper on the influent side of the trash racks
        - ``hopper_slope (float)``: The slope of the hopper walls (optional, defaults to 1)
        - ``tRack_number (float)``: The number of trash racks (optional, defaults to 2)
        - ``tRack_spacing (float * u.m)``: the space needed between trash racks (optional, defaults to 12 cm)
        - ``track_diam (float * u.m)``: the diameter of holes in the trash rack (recommended, defaults to 2 mm)
        - ``sdr (string)``: Standard dimension ratio (optional, dafults to 'sdr41')
        - ``drain_nd (float * u.inch)``: The diameter of the drain pipe (optional, defaults to 3 inch)
        - ``submersion_depth (float * u.m)``: the distance that the first orifice on the influent pipe is submerged below the surface (optional, defaults to ___ m)
  """
  def __init__(self, **kwargs):
      self.q = (60 * u.L/u.s).to(u.m**3/u.s)
      self.temp = 20 * u.degC
      self.diam_p = (2.5 * u.mm).to(u.m)
      self.lfom_od = (10 * u.inch).to(u.m) # check if this is an actual pipe size. What is used in Gracias?
      self.lfom_hl = (30 * u.cm).to(u.m)
      self.flocc_gt = 37000
      self.flocc_hl = (40.0 * u.cm).to(u.m)
      self.flocc_widthMax = 1.08 * u.m
      self.flocc_widthMin = 0.45 * u.m
      self.flocc_depth = 2.0 * u.m
      self.dividing_wall_thickness = (15 * u.cm).to(u.m)
      self.sed_length = 7.1 * u.m
      self.fab_s = (5 * u.cm).to(u.m)
      self.hopper_LWratio = 1
      self.influentHopper_LWratio = 1
      self.hopper_slope = 1 # corresponds to a 45 deg angle, the angle of repose for sand when wet
      self.tRack_number = 2
      self.tRack_spacing = (12 * u.cm).to(u.m) # large enough to fit a hand in between the trash racks
      self.tRack_diam = (2 * u.mm).to(u.m) # corresponds to Number 10 SS mesh
      self.sdr = 'sdr41'
      self.drain_nd = 3 * u.inch
      self.drain_pipe = Pipe()
     # self.subcomponents = [self.drain_pipe]
      #self.submersion_depth = * u.m
      

     # super().__init__(**kwargs) # no idea what this does but it's in all the other code
     # self.set_drain_pipe()
      #super().set_subcomponents()

  @property
  def et_depth(self):
      et_depth = self.flocc_depth + self.lfom_hl
      return et_depth

  @property
  def grit_area(self):
      """the required area of the grit chamber"""
      grit_area = 0 * u.m**2
      if self.diam_p < self.tRack_diam: # there is no need for a grit chamber if the desired particle size will be trapped by the trash rack 
         rho_H20 = ac.density_water(self.temp)
         rho_sand = 2650 *u.kg/u.m**3
         nu = ac.viscosity_kinematic_water(self.temp)
         v_c = ((rho_sand - rho_H20)*ac.GRAVITY*diam_p**2)/(18*nu*rho_H20)
         Re = v_c*diam_p/nu
         if Re > 2: # recalculate for transition flow (MWH textbook uses 2 as laminar boundary)
            v_c = (((rho_sand - rho_H20)*ac.GRAVITY*diam_p**1.6)/(13.9*rho_H20*nu**0.6))**(1/1.4) 
            Re = v_c*diam_p/nu
            grit_area = q/v_c
      return grit_area

  @property
  def etLFOM_length(self):
      """The length needed by the LFOM in the entrance tank"""
      etLFOM_length = 3 * self.lfom_od
      return etLFOM_length

  @property
  def tRack_length(self):
      """the length of the entrance tank needed for the trash racks"""
      tRack_length = self.tRack_spacing * (self.tRack_number - 1)
      return tRack_length

  @property
  def flocc_area(self):
      """the required area of the flocculator"""
      vel_grad_avg = ((ac.GRAVITY * self.flocc_hl) / (ac.viscosity_kinematic_water(self.temp) * self.flocc_gt)).to(u.s ** -1)
      retention_time = (self.flocc_gt / vel_grad_avg).to(u.s)
      flocc_volume = (self.q * retention_time).to(u.m ** 3)
      flocc_area = flocc_volume/self.flocc_depth
      return flocc_area   

  def etFlocc_specs(self): # do we need a mechanism to choose an even or odd number of channels?
      """Calculates the length and width of the entrance tank, the width of the flocculator, and the number of channels.
      This algorithm selects the minimum number of channels (to save concrete) and it could be modified to select the maximum number of channels (to save plastic)"""
      n = [1,2,3,4,5,6] # how big should we make this matrix though? What if you have a really big plant, should it stop at 6?, could also do n = n + 1 in loop instead
      etInfluent_LWratio = 5 # comes from hydraulics. Want jet to expand to half the width of the entrance tank
      i = 0
      et_width = 0
      flocc_width = 0
      found = False
      while found == False and i < len(n):
            n_channels = n[i]
            a = etInfluent_LWratio
            b = self.etLFOM_length + self.tRack_length + self.dividing_wall_thickness - n_channels*self.sed_length  
            c = self.flocc_area + self.grit_area
            W1 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
            W2 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
            if W1 >= self.flocc_widthMin and W1 <= self.flocc_widthMax: # if W1 and W2 are both in the deired range, this returns the larger width. This can be changed to return the smaller width, or both of them.
              found = True
              et_width = W1
              flocc_width = W1
            elif W2 >= self.flocc_widthMin and W2 <= self.flocc_widthMax:
              found = True
              et_width = W2
              flocc_width = W2
            i = i + 1

      # determine if the entrance tank gets its own channel
      etInfluent_length = etInfluent_LWratio * et_width
      grit_length = self.grit_area/et_width
      et_length = etInfluent_length + self.tRack_length + grit_length + self.etLFOM_length
      #print('TEST - et_width',et_width)
      #print('TEST - etInfluent_length',etInfluent_length)
      #print('TEST - tRack_length',self.tRack_length)
      #print('TEST - grit_length',grit_length)
      #print('TEST - etLFOM_length',self.etLFOM_length)
      #print('TEST - ET_length',et_length)

      if et_length > self.sed_length or (self.sed_length - et_length) < self.flocc_widthMin: # rename this last variable. Then ET gets it own channel
         et_length = self.sed_length
         # re-design the flocculator
         flocc_nChan = np.ceil(self.flocc_area/(self.sed_length * self.flocc_widthMax)) # make this to an odd number
         flocc_width = self.flocc_area/(self.sed_length*flocc_nChan)

         # re-design the entrance tank (n = 1)
         a = etInfluent_LWratio
         b = self.etLFOM_length + self.tRack_length - self.sed_length
         c = self.grit_area
         W1 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
         W2 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
         if W2 >= self.flocc_widthMin: # rename this last variable, try the smaller width first
            et_width = W2
            flocc_width = W2
         elif W1 >= self.flocc_widthMin:
              et_width = W1
              flocc_width = W1
         #else is it possible that there will be no solution?
         n_channels = flocc_nChan + 1
      specs = [et_width, et_length, flocc_width, n_channels]
      return specs

  @property
  def et_width(self):
      """the width of the entrance tank"""
      et_width = self.etFlocc_specs()[0]
      return et_width

  @property
  def et_length(self):
      """the length of the entrance tank"""
      et_length = self.etFlocc_specs()[1]
      return et_length

  @property
  def flocc_width(self):
      """the width of a floccualtor channel"""
      flocc_width = self.etFlocc_specs[2]
      return flocc_width

  @property
  def n_channels(self):
      """the total number of entrance tank and flocculator channels"""
      n_channels = self.etFlocc_specs[3]
      return n_channels

  @property
  def etInfluent_length(self): # Consider returning this value in etFlocc_specs since it is already calculated there
      """The length of the entrance tank on the influent side of the trash racks"""
      etInfluent_LWratio = 5 # comes from hydraulics. Want jet to expand to half the width of the entrance tank
      etInfluent_length = etInfluent_LWratio * self.et_width
      return etInfluent_length

  @property
  def tRack_width(self):
      """the width of the trash rack"""
      tRack_width = self.et_width 
      return tRack_width

  @property
  def grit_length(self): # Consider returning this value in etFlocc_specs
      """the length of the entrance tank containing hoppers on the effluent side of the trash racks"""
      grit_length = self.grit_area/self.et_width # does this need to be self.et_width?
      return grit_length

  @property
  def hopper_width(self):
      """the width of one hopper"""
      hopper_width =  self.et_width
      return hopper_width

  @property
  def gritHopper_length(self): 
      """Calculates the length of hoppers on the effluent side of the trash rack"""
      gritHopper_length = 0 * u.m
      if self.grit_area != 0:
         gritHopper_length = self.hopper_LWratio * self.hopper_width
         n_hoppers = np.floor(self.grit_length/gritHopper_length)
         # redistribute extra space to each hopper equally
         extraLength = (self.grit_length - n_hoppers*gritHopper_length)/n_hoppers # the extra length added to each hopper to fill the entire grit chamber
         gritHopper_length = gritHopper_length + extraLength
         hopper_LWratioActual = gritHopper_length/self.hopper_width
      return gritHopper_length

  @property
  def gritHopper_number(self): # is there a better way to do this? (this variable is already defined in gritHopper_length)
      """the number of hoppers on the effluent side of the trash racks"""
      gritHopper_number = 0
      if self.grit_area != 0:
         gritHopper_number = self.grit_length/self.gritHopper_length
      return gritHopper_number

  def set_drain_pipe(self): # okay to use flocc_depth in this function??
      """Calculates the inner diameter of the entrance tank drain pipes"""
      #drain_pipe_k_minor = ac.PIPE_ENTRANCE_K_MINOR + ac.PIPE_EXIT_K_MINOR + ac.EL90_K_MINOR
      #nu = ac.viscosity_kinematic_water(self.temp) # should I make viscosity its own @property since it is also used in another function?
      #drain_id = ac.diam_pipe(self.q,self.flocc_depth,self.flocc_depth,nu,ac.PVC_PIPE_ROUGH,drain_pipe_k_minor)
      #self.drain_pipe = Pipe(id=drain_id,k_minor=drain_pipe_k_minor,spec=self.sdr)
      self.drain_pipe = Pipe(self.drain_nd,self.sdr)

  @property
  def drainfloor_length(self):
      """the side length of the drain floor, assuming a square geometry, centered in the hopper"""
      drainfloor_length = (self.drain_pipe.od).to(u.m) + 2 * self.fab_s
      return drainfloor_length

  @property
  def gritHopper_depth(self):
      """The vertical distance from the bottom of the entrance tank to the top of a hopper on the effluent side of the trash rack"""
      gritHopper_depth = 0 * u.m
      if self.grit_area != 0:
         gritHopper_lengthWallLong = 0.5*(self.gritHopper_length - self.drainfloor_length) # the distance between the drain floor and the edge of the hopper in the longitudinal direction
         gritHopper_depth = self.hopper_slope * self.gritHopper_lengthWallLong
      return gritHopper_depth

  @property
  def influentHopper_length(self): 
      """Calculates the length of a hopper on the influent side of the trash rack"""
      influentHopper_length = self.influentHopper_LWratio * self.hopper_width
      n_hoppers = np.floor(self.etInfluent_length/influentHopper_length)
      # redistribute extra space to each hopper equally
      extraLength = (self.etInfluent_length - n_hoppers*influentHopper_length)/n_hoppers # the extra length added to each hopper to fill the entire influent area
      influentHopper_length = influentHopper_length + extraLength
      return influentHopper_length

  @property
  def influentHopper_number(self): # is there a better way to do this? (this variable is already defined in influentHopper_length)
      """the number of hoppers on the influent side of the trash racks"""
      influentHopper_number = self.etInfluent_length/self.influentHopper_length
      return influentHopper_number 

  @property
  def influentHopper_depth(self):
      """The vertical distance from the bottom of the entrance tank to the top of a hopper on the influent side of the trash rack"""
      influentHopper_lengthWallLong = 0.5*(self.influentHopper_length - self.drainfloor_length) # the distance between the drain floor and the edge of the hopper in the longitudinal direction
      influentHopper_depth = self.hopper_slope * influentHopper_lengthWallLong
      return influentHopper_depth

  #@property # I need to fix/replace this property
  #def tRackSupportHeight(self):
  #    tRackSupportHeight = (np.max([self.gritHopper_depth.magnitude,self.influentHopper_depth.magnitude,self.flocc_widthMin.magnitude]))*u.m
  #    return tRackSupportHeight

  @property
  def tRack_depth(self): # this can be adjusted if the trash rack is too big to lift
      """The height of the trash rack"""
      if self.grit_area == 0:
         tRack_depth = self.et_depth - (np.max([self.gritHopper_depth.magnitude,self.influentHopper_depth.magnitude,self.flocc_widthMin.magnitude]))*u.m
         #tRack_depth = self.et_depth - self.tRackSupportHeight
      else:
         tRack_depth = self.et_depth - np.max([self.gritHopper_depth.magnitude,self.influentHopper_depth.magnitude])
      return tRack_depth
  
  #@property
  #def orifice_number(self):
  #    """The number of orifices on the influent pipe"""
  #    orifice_number = self.etInfluent_length/(self.etInfluent_LWratio * (self.et_depth - self.submersion_depth))
  #    return orifice_number
  
 # @property
 # def orifice_spacing(self): # can be rewritten as center-center distance if needed
 #   """The edge-edge distance between orifices"""
 #   orifice_spacing = (self.et_depth - self.submersion_depth - self.orifice_number*self.orifice_diam)/self.orifice_number
 #   return orifice_spacing

#### Code below has not been updated yet #####

  #@property
  #def etObstacle_length(self):

ET = EntranceTank()
print('The width of the entrance tank is ', ET.et_width)
print('The length of the entrance tank is ',ET.et_length)
print('The depth of the entrance tank is ',ET.et_depth)  
print('The length needed for the LFOM is ',ET.etLFOM_length)  
print('The length needed for the trash racks is ', ET.tRack_length)  
print('The length needed for the influent region is ',ET.etInfluent_length) 
print('The width of the entrance tank is ',ET.hopper_width)
print('The length of the grit chamber is ',ET.grit_length)
print('The length of a grit hopper is ',ET.gritHopper_length)
print('The length of the drain floor is ',(ET.drainfloor_length).to(u.m))
print('The depth of the grit hopper is ',ET.gritHopper_depth)
print('The length of an influent hopper is ',ET.influentHopper_length)
print('The depth of the influent hopper is',ET.influentHopper_depth)
print('The height of the trash rack is ',ET.tRack_depth)
print('The dividing wall thickness is ', ET.dividing_wall_thickness)
print('The number of hoppers on the influent side of the trash racks is ', ET.influentHopper_number)
print('The number of hoppers on the effluent side of the trash racks is ', ET.gritHopper_number)

 
 


The width of the entrance tank is  0.6103 meter
The length of the entrance tank is  3.933 meter
The depth of the entrance tank is  2.3 meter
The length needed for the LFOM is  0.762 meter
The length needed for the trash racks is  0.12 meter
The length needed for the influent region is  3.051 meter
The width of the entrance tank is  0.6103 meter
The length of the grit chamber is  0 meter
The length of a grit hopper is  0 meter
The length of the drain floor is  0.1213 meter
The depth of the grit hopper is  0 meter
The length of an influent hopper is  0.6103 meter
The depth of the influent hopper is 0.2445 meter
The height of the trash rack is  1.85 meter
The dividing wall thickness is  0.15 meter
The number of hoppers on the influent side of the trash racks is  5 dimensionless
The number of hoppers on the effluent side of the trash racks is  0


/usr/local/lib/python3.6/dist-packages/pint/numpy_func.py:295: RuntimeWarning: invalid value encountered in sqrt
  result_magnitude = func(*stripped_args, **stripped_kwargs)


In [ ]:

      #self.q = (60 * u.L/u.s).to(u.m**3/u.s)
      #self.temp = 20 * u.degC
      #self.diam_p = (2.5 * u.mm).to(u.m)
      #self.lfom_od = (10 * u.inch).to(u.m) # check if this is an actual pipe size. What is used in Gracias?
      #self.lfom_hl = (30 * u.cm).to(u.m)
      #self.flocc_gt = 37000
      #self.flocc_hl = (40.0 * u.cm).to(u.m)
      #self.flocc_widthMax = 1.08 * u.m
      #self.flocc_widthMin = 0.45 * u.m
      #self.flocc_depth = 2.0 * u.m
      #self.sed_length = 7.1 * u.m
      #self.fab_s = (5 * u.cm).to(u.m)
      #self.hopper_LWratio = 1
      #self.hopper_slope = 1 # corresponds to a 45 deg angle, the angle of repose for sand when wet
      #self.tRack_number = 2
      #self.tRack_spacing = (12 * u.cm).to(u.m) # large enough to fit a hand in between the trash racks
      #self.tRack_diam = (2 * u.mm).to(u.m) # corresponds to Number 10 SS mesh
      #self.sdr = 'sdr41'
      #self.drain_nd = 3 * u.inch
      #self.drain_pipe = Pipe() # Pipe is imported with AguaClara right??
      #self.subcomponents = [self.drain_pipe]
      #self.submersion_depth = * u.m


# Python Calculations

In [ ]:
!pip install aguaclara
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import numpy as np

# global variables
q = (60 * u.L/u.s).to(u.m**3/u.s)
T = 20 * u.degC
rho_H20 = ac.density_water(T)
rho_p = 2650 *u.kg/u.m**3 # density of sand
nu = ac.viscosity_kinematic_water(T) # kinematic viscosity
mu = nu*rho_H20 # dynamic viscosity of 
w_flocculator = 0.60 * u.m # the width of one flocculator at Gracias (inner width ie. does not include thickness of the concrete)
length_flocculator = 7.10 * u.m # the length of one flocculator channel at Gracias (inner length ie. does not include thickness of the concrete)
depth_flocculator = 2.17 * u.m  # at Gracias (flocculator design code say 2.08)




     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 204kB 19.2MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.3-cp36-none-any.whl size=99459 sha256=22ddc9ec328cd70ef44b8d43ffdcd19395d4750ed6caf88405d84aefe389468c
  Stored in directory: /root/.cache/pip/wheels/da/f3/b6/a4648b3b3bc239e7953e74a2dee9f21c07d1b1e1f65cb86311
Successfully built aguaclara


## Analyze the Kinetic Energy in the Entrance Tank at the Gracias, Honduras Plant

### Determine if the trash racks will help dissipate the influent kinetic energy

In [ ]:
# trash racks made of #10 SS round-wire, 2 mm diameter mesh

n_tRack = 2 # number of trash racks
n_tRackSlots = n_tRack + 1 # always need an empty slot so that the order of trash racks can be rearranged 
s_tRack = (8 * u.cm).to(u.m) # center-center distance between trash racks (8 cm is an arbitrary guess)
d = (2 * u.mm).to(u.m) # the size of holes in the mesh (probably square holes, not round holes )
d_openings = (10/u.inch).to(1/u.m) # density of opening in #10 mesh
#n_orifices = # number of orifices per trash rack
#d_orifices = # orifice diameter
#thickness_tRack = # depends on material probably
width_tRack = width_ET
height_tRack = depth_flocculator

#h_L through a wire mesh equation from 4520 ET design
h_L 

### Calculate the kinetic energy and velocity exiting the influent pipe at Gracias


In [ ]:
ID = 0.2 * u.m # inner diameter of influent pipe at Gracias
a = 0.25*np.pi*ID**2
v_influent = q/a
KE_influent = v_influent**2/(2*ac.GRAVITY)

print('The velocity exiting the influent pipe is ', v_influent)
print('The kinetic energy exiting the influent pipe is ', KE_influent.to(u.cm))

The velocity exiting the influent pipe is  1.91 meter / second
The kinetic energy exiting the influent pipe is  18.6 centimeter


### Calculate Ke = v^2/2g through the plate settlers at Gracias


In [ ]:
# calculate the kinetic energy through the plate settlers at Gracias

s = 0.0250 * u.m # edge-to-edge distance between plate settlers (ie. does not include plate thickness)
n = 24 # number of plate settlers in entrance tank
w = 0.6 * u.m # plate settler width
area = s*(n+1)*w
v = q/area
Ke = v**2/(2*ac.GRAVITY)

print('The hypothetical velocity through the plate settlers assuming uniform flow is ', v)
print('The kinetic energy through the plate settlers at Gracias is ', Ke.to(u.cm))

The hypothetical velocity through the plate settlers assuming uniform flow is  0.16 meter / second
The kinetic energy through the plate settlers at Gracias is  0.1305 centimeter


### Calculate the velocity through the channel leading to the LFOM

In [ ]:
# calculate the velocity through the channel leading to the LFOM

d = 0.6278 * u.m # depth of water immediately after the plate settlers measured to the top hole on the LFOM (ie. assuming holes are completely covered at 60 L/s flow)
w = 0.6 * u.m # width of ET
area_LFOMchan = d*w
v_LFOMchan = q/area_LFOMchan
Ke_LFOMchan = v_LFOMchan**2/(2*ac.GRAVITY)

print('The velocity in the channel leading to the LFOM is ', v_LFOMchan)
print('The kinetic energy in the channel leading to the LFOM ', Ke_LFOMchan.to(u.cm))

The velocity in the channel leading to the LFOM is  0.1593 meter / second
The kinetic energy in the channel leading to the LFOM  0.1294 centimeter


**Reflection:** It makes sense that neither the velocity or Ke change very much in these two calculations, because the cross-sectional area for flow didn't really change, and v = q/a. The Ke through the plate settlers is well below 5 cm (5 cm would be considered very high)

## Calculate the Required Length of the Entrance Tank

*Assume that the entrance tank has the same width as the flocculator and that the plant is running at full capacity. Calculations will focus on the inlet channel of the sedimentation tank, since high shear around baffles in the flocculator makes it less likely that particles will settle there.*

**Strategy**

1. Determine the diameter of the smallest particle that needs to be removed by the entrance tank.
    - **Approach 1:** Use a critical particle size of 0.1 mm recommended by Kawamura (pg. 416)
    - **Approach 2:** Calculate the diameter of the largest particle that will be scoured in the inlet channel of the sedimentation tank by rearranging the [equation](https://github.com/AguaClara/CEE4520/blob/master/Lectures/Extras/Sedimentation%20Extras.pptx) (slides 15-18) for scouring velocity:$$V_{min} = \sqrt{\frac{2}{3}\frac{\sqrt{Re_x}}{0.332}\frac{\rho_{p} - \rho_{H20}}{\rho_{H20}}\ g\ d_p\ tan\theta}$$
    - **Approach 3:** Calculate the shear on the bottom of the inlet channel of the sedimentation tank and use a modified Shields diagram to determine the critical particle size. 
1. Use the appropriate Stokes Law for laminar or turbulent flow to determine the corresponding critical velocity of the particle.

   $$**insert \ equations**$$
1. Calculate the required plan-view area of the entrance tank
   $$**insert \ equations**$$

### **Approach 1:** Use the 0.1 mm recommended particle diameter provided by Kawamura (pg. 416)

In [ ]:
d_p = (0.1 * u.mm).to(u.m) # minimum grit diameter to be removed according to Kawamura
v_c = ((rho_p - rho_H20)*ac.GRAVITY*d_p**2)/(18*nu*rho_H20)
print('The cricitcal velocity is ',v_c.to(u.mm/u.s))

# check that flow is in laminar regime for Stokes' law to be valid
Re = v_c*d_p/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area = q/v_c
length_ET = area/w_flocculator
print('The required entrance tank length is ', length_ET)

The cricitcal velocity is  6.919 millimeter / second
The Reynolds number is  0.5322 dimensionless
The required entrance tank length is  14.45 meter


**Reflection:** The calculated critical velocity is 8.984 mm/s, which is very close to the 8 mm/s that was suggested by Monroe. The Reynolds number is well within the laminar regime so using Stokes' law was an appropriate decision. The entrance tank requires a length of 11.13 meters. For comparison, the length of one treatment train at Gracias (sed tanks + flocculators) is approxiamtely 15 m, and the width of one treatment train is about 7 m. Therefore, trying to capture such small particles takes up a considerable amount of space, and may drive up materials and construction costs. Additionally, an entrance tank of that size my block plant operator access to important parts of the plant, such as the first four baffles in the flocculator, where plant operators like to observe flocs. 

### **Approach 2:** Calculate the scouring velocity in the inlet channel of the sedimentation tanks






For open-channel flow, the [equation](https://github.com/AguaClara/CEE4520/blob/master/Lectures/Extras/Sedimentation%20Extras.pptx) (slides 15-18) describing scouring velocity is:

$$V_{min} = \sqrt{\frac{2}{3}\frac{\sqrt{Re_x}}{0.332}\frac{\rho_{p} - \rho_{H20}}{\rho_{H20}}\ g\ d_p\ tan\theta}$$

where

$Re_x$ = the Reynold's number based on the distance along the channel. Note that the lowest amount of shear occurs where the boundary layer switches from laminar to turbulent (ie. $Re_x$ = 500,000) (slide 15).

$\rho_{p}$ = density of the particle

$\rho_{H20}$ = density of the water

$g$ = acceleration due to gravity

$d_p$ = the particle diameter

$\theta$ = a parameter that is approximated from Shields Diagram (slide 18), assuming turbulent flow
     $$\theta =  tan^{-1}(0.056* \frac{3}{2}) = 4.8^\circ$$ 

Knowing the average velocity in the inlet channel, this equation can be rearranged to calculate the size of the largest particle that will be scoured. The inlet channel is designed with a sloped bottom such that it has a relatively constant velocity along the length of the channel. The simplest place to calculate the average velocity is the point where water enters the channel.  


In [ ]:
# calculate the average velocity in the inlet channel to the sedimentation tanks, assuming that it is relatively constant due to sloped bottom geometry
theta = (4.8*u.deg).to(u.rad)
w_chan = 0.5 * u.m # width of the inlet channel to the sedimentation tanks
h_chan = 0.156 * u.m # the height of the inlet channel measured at the influent end
a_chan = w_chan*(h_chan - 0.1 * u.m) # The inlet channel to the sedimentation tanks have a 10 cm freeboard
v_chan = q/a_chan
print('The velocity in the influent channel of the sedimentation tank is ', v_chan)

# rearrange the scouring velocity equation to calculate the critical particle diameter
dp_2 = (3*0.332*rho_H20*v_chan**2)/(2*500000**0.5*(rho_p - rho_H20)*ac.GRAVITY*np.tan(theta))
print('The largest diameter particle that will be scoured by the inlet channel of the sedimentation tanks is ', dp_2.to(u.mm))

# calculate the terminal velocity of the particle using Stokes' Law assuming laminar flow
vc_2 = ((rho_p - rho_H20)*ac.GRAVITY*dp_2**2)/(18*nu*rho_H20)
print('The cricitcal velocity is ',vc_2.to(u.mm/u.s))

# check that flow is in laminar regime for Stokes' law to be valid
Re = vc_2*dp_2/nu
print('The Reynolds number is ', Re)
print('The calculation needs to repeated using Stokes Law for transition flow')

# calculate the terminal velocity of the particle assuming the transition/turbulent regime
v_c_transition = (((rho_p - rho_H20)*ac.GRAVITY*dp_2**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4)
print('The critical velocity assuming transitional flow is ', v_c_transition)

# check that flow is in the transition/turbulent regime
Re = v_c_transition*dp_2/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area2 = q/v_c_transition
length_ET2 = area2/w_flocculator # assume ET is same width as the flocculator
print('The entrance tank length is ', length_ET2)

The velocity in the influent channel of the sedimentation tank is  2.143 meter / second
The largest diameter particle that will be scoured by the inlet channel of the sedimentation tanks is  2.379 millimeter
The cricitcal velocity is  3915 millimeter / second
The Reynolds number is  7165 dimensionless
The calculation needs to repeated using Stokes Law for transition flow
The critical velocity assuming transitional flow is  0.3728 meter ** 1 / second
The Reynolds number is  682.2 dimensionless
The entrance tank length is  0.2682 meter ** 1


In [ ]:
# This is scrap code: I'm backchecking this calculation with current Gracias dimensions
Length_ET = (16 * u.cm).to(u.m)
area = Length_ET*w_flocculator
v_c = q/area
d_p = (v_c**(1.4)*(13.9*mu**0.6*rho_H20**0.4)/((rho_p - rho_H20)*ac.GRAVITY))**(1/1.6) # uses transition equation
Re = v_c*d_p/nu

print('the critical particle size is ',d_p)
print('Re is ', Re)

# the holes on the trash rack appear to be about 12.8 (rounds to 13 mm in diameter), so any particles we are talking about here would pass through
# so theoretically, the current entrance tank at Gracias should be able to capture a 3.388 mm particle

the critical particle size is  0.003388 meter ** 1
Re is  2110 / meter ** 2.22045e-16


**Reflection:** The critical particle size is 2.373 mm and the required entrance tank length is 24 cm. We should be skeptical of these results, because the entrance tank at Gracias is already longer than 24 cm long, and plant operators are observing grit in both the flocculator and inlet to the sedimentation tanks. 

### **Aproach 3:** Calculate the bottom shear in the inlet channel and use a modified Shields Diagram

**Bottom Shear Stress**

We will assume a turbulent boundary layer on the bottom of the inlet channel to the sedimentation tanks, since achieving laminar flow in a water treatment plant takes considerable effort. Wall shear stress, $\tau_w$, in the turbulent regime can be modeled by the following [Equation:](http://www-mdp.eng.cam.ac.uk/web/library/enginfo/aerothermal_dvd_only/aero/fprops/introvisc/node8.html)

$$\tau_w = \frac{1}{2}\rho_{H20} U_ {\infty}^2 C_f $$

where 

$\rho_{H20}$ = density of water

$U_ {\infty}$ = free stream speed. 

$C_f$ = the skin friction

<br/>

In the turbulent regime,

$$C_f = \frac{0.0594}{Re_x^{0.2}}$$ 

where

$Re_x$ = the Reynolds number with respect to the distance along the channel

The shear is the lowest when the boundardy layer switches from laminar to turbulent. Thereofre $Re_x$ will be assumed to be 500,000 [(slide 15)](https://github.com/AguaClara/CEE4520/blob/master/Lectures/Extras/Sedimentation%20Extras.pptx).

<br/>

**Shields Diagram**

Shields diagram represents the empirical relationship between two dimensionless parameters that describe the incipient movement of particles ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), pg. 268): 

1) *The 'Shear' or 'Boundary' Reynolds Number*

$$Re_* = \frac{u_*D}{\nu}$$

where

$u_* = \sqrt{\frac{\tau_o}{\rho}}\ \ $"shear velocity"

$\rho$ = density of the fluid

D = particle diameter

$\nu$ = kinematic viscosity

<br/>

2) *Shields Parameter* ([Hydraulic](https://github.com/monroews/Hydraulics/raw/master/10_Channel_Design.pptx), slides 19-23)

$$\frac{\tau_o}{\Delta\rho gD} = \frac{2}{3}tan\theta$$

where

$\tau_o$ = shear stress

$\Delta \rho$ = the difference in density between the particle and water

$D$ = particle diameter

*Side note: the $\frac{2}{3}$ fraction on the right hand side may also depend on geometrical and dynamical effects ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), pg. 267) and multiplying the numerator and denominator by $D^2$ gives a ratio of fluid force on the particle to the weight of the particle (pg. 269).*

The original version of Shields Diagram is not particularly useful for determining the critical particle size that will be scoured in the inlet channel, because both shear stress and particle diameter occur as variables on both the x and y axis. 

However, Shields diagram can be 'recast' so that only one axis contains the variable $\tau_o$ and only the other axis contains the particle diameter D. These new axis are as follows ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), pg. 275): 

$$\tau_o(\rho /(g\Delta\rho)^2\mu^2)^{1/3}\ \ \  and\ \  \ D(\rho (g\Delta\rho)/\mu^2)^{1/3}$$

Figure 1 shows an updated version of Shields diagram reframed in terms of shear velocity and particle diameter at $20 ^\circ C$ ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), pg. 277). Note that the axis do not look like the ones described above, because constant values at the given temperature such as the acceleration due to gravity and dynamic viscosity have already been plugged in. 

![Shields Diagram](https://github.com/Emily-Wood/AguaClara/blob/master/Updated_Shields_Diagram%20.PNG?raw=true)


**Figure 1**: Shields diagram recast in terms of shear velocity and particle diameter at $20 ^\circ C$ ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), page 277)

Using the calculated bottom shear stress, the modified Shields Diagram can be used to determine the critical particle size. 

**Sources:** 
1. Chapter 9 [Introduction to Fluid Motion, Sediment Transport and Current Generated Sedimentary Structures](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf)
1. [Hydraulic Engineering: Channel Design Powerpoint](https://github.com/monroews/Hydraulics/raw/master/10_Channel_Design.pptx) (slides 19-23)









In [ ]:
# calculate the average velocity in the inlet channel of the sedimentation tanks (ie. "free stream speed")
L_chan = 12 * u.m
w_chan = 0.5 * u.m
h_chan = 0.156 * u.m # the height of the channel at the influent end
a_chan = w_chan*(h_chan - 0.1 * u.m) # The inlet channel to the sedimentation tanks have a 10 cm freeboard
v_chan = q/a_chan

#Re_x = 500000
Re_x = v_chan*L_chan/nu # choose the far end of the channel for a conservative estimate of tau
print('The Reynolds number with respect to the channel length is ',Re_x)
C_f = 0.0594/(Re_x**0.2)
tau = 0.5*rho_H20*v_chan**2*C_f

u_star = (tau/rho_H20)**0.5
print('The shear velocity is ', u_star.to(u.cm/u.s))

# According to the recast Shields Diagram, the corresponding critical particle diameter is approximately 1.3 cm
dp_3 = (7.5 * u.mm).to(u.m)
print('Using the modified Shields diagram, this corresponds to a particle diameter of ', dp_3.to(u.mm))

# Calculate the terminal velocity of the particle assuming transitional/turbulent flow
v_c_transition2 = (((rho_p - rho_H20)*ac.GRAVITY*dp_3**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4) 
print('The cricitcal velocity is ', v_c_transition2)

# check that flow is in the transitional/turbulent regime
Re = v_c_transition2*dp_3/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area3 = q/v_c_transition2
length_ET3 = area3/w_flocculator
print('The entrance tank length is ', length_ET3)

The Reynolds number with respect to the channel length is  1.978e+07 dimensionless
The shear velocity is  6.882 centimeter / second
Using the modified Shields diagram, this corresponds to a particle diameter of  7.5 millimeter
The cricitcal velocity is  1.385 meter ** 1 / second
The Reynolds number is  7990 dimensionless
The entrance tank length is  0.07221 meter ** 1


**Reflection:** With a Reynolds number of 500,000 the shear velocity is 9.942 cm/s. The corresponding critical particle diameter of 1.3 cm is clearly not a physically reasonable result. A piece of grit this large probably would not even make it into the plant, and would certainly be caught by the trash racks. Perhaps the equations selected to model the bottom shear in the inlet channel are not applicable to this situation, or the wrong Reynolds number was selected. It is also really hard to read the recast Shields diagram without gridlines - a small amount of reading error could lead to an order of magnitude difference in particle diameter. The initial assumption of this calculation should be checked, and a different method for approximating the shear stress on the bottom of the inlet channel could be considered. 

To troubleshoot this approach, I first tried calculating the Reynold's number for the channel directly instead of assuming that it is 500,000. I used the total channel length as the important length, because it would lead to the most conservative measure of shear stress (and less shear means that it is more difficult to scour particles). This led to a new Reynolds number of 25,620,000 (This number is really high, I'm not sure if it is reasonable for the open channel flow in this application) and a shear velocity of 6.707 cm/s. These numbers produce an approximate critical particle size of 7.5 mm, and an entrance tank length of 6.5 cm. This is still a very large particle size. 

Approach 3 is also less reliable than approach 2, because it relies on reading an empirical graph precisely rather than calculating particle size directly using first principles. 




## Old Python Code

In [ ]:
#### Old Code 7/22/20 ####

"""The entrance tank of an AguaClara water treatment plant"""

!pip install aguaclara
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import numpy as np

class EntranceTank(Component):
  """Design an AguaClara plant's entrance tank. 

     An entrance tank's design relies on the LFOM's and the floccualtor's design in
     the same plant, but assumed/default values may be used to design an entrance
     tank by itself. To design these components in tandem, use :class:`aguaclara.design.ent_floc.EntTankFloc`.

     Design Inputs:
        - ``q (float * u.L / u.s)``: Flow rate (recommended, defaults to 60 L/s)
        - ``temp (float * u.degC)``: Water temperature (recommended, defaults to 20°C)
        - ``diam_p (float * u.mm)``: The smallest particle that needs to be captured by the entrance tank grit chamber (optional, defaults to 2.5 mm)
        - ``lfom_nd (float * u.inch)``: The LFOM's nominal diameter (recommended,defaults to 10 inch")
        - ``flocc_area (float * u.m**2)``: The required area of the flocculator (recommended, defaults to ____ m**2)
        - ``flocc_widthMax (float * u.m)``: The maximum width of a flocculator channel based on the polycarbonate sheets used (optional, defaults to 1.08 m)
        - ``flocc_widthMin (float * u.m)``: The minimum width of a flocculator based on average human width (optional, defaults to 0.45 m) 
        - ``sed_length (float * u.m)``: the length of a sedimentation tank plus the width of the influent and effluent sedimentation channels (recommended, defaults to ___ m)
        - ``fab_s(float * u.cm)``: The space needed for a person to remove the drain pipe (optional, defaults to 5 cm)
        - ``hopper_LWratio (float)``: The length:width ratio of a hopper. Used to manipulate the tradeoff between hopper depth and number of hoppers (optional, defaults to 1)
        - ``hopper_slope (float)``: The slope of the hopper walls (optional, defaults to 1)
        - ``flocc_depth (float)``: The depth of the flocculator (recommended, defaults to 2 m)
        - ``sdr (string)``: Standard dimension ratio (optional, dafults to 'sdr41')
  """
  def __init__(self, **kwargs):
      self.q = (60 * u.L/u.s).to(u.m**3/u.s)
      self.temp = 20 * u.degC
      self.diam_p = (2.5 * u.mm).to(u.m)
      self.lfom_nd = (10 * u.inch).to(u.m) # is this an actual pipe size. What is used in Gracias? # why are we using nd and not OD?
      self.flocc_area = 
      self.flocc_widthMax = 1.08 * u.m
      self.flocc_widthMin = 0.45 * u.m
      self.sed_length = 
      self.fab_s = (5 * u.cm).to(u.m)
      self.hopper_LWratio = 1
      self.hopper_slope = 1 # corresponds to a 45 deg angle, the angle of repose for sand when wet
      self.tRack_diam = (2 * u.mm).to(u.m)
      self.flocc_depth = 2.0 * u.m
      self.sdr = 'sdr41'

      self.drain_pipe = Pipe() # Pipe is imported with AguaClara right??
      self.subcomponents = [self.drain_pipe]

      super().__init__(**kwargs) # no idea what this does but it's in all the other code
      self.set_drain_pipes()
      super().set_subcomponents()

      # Ok to use baffle_spacing as a constraint? define SF to go with this

  @property
  def grit_area(self):
      """the required area of the grit chamber"""
      grit_area = 0 * u.m**2
      if self.diam_p < self.tRack_diam: # there is no need for a grit chamber if the desired particle size will be trapped by the trash rack 
         rho_H20 = ac.density_water(self.temp)
         rho_sand = 2650 *u.kg/u.m**3
         nu = ac.viscosity_kinematic_water(self.temp)
         v_c = ((rho_sand - rho_H20)*ac.GRAVITY*diam_p**2)/(18*nu*rho_H20)
         Re = v_c*diam_p/nu
         if Re > 2: # recalculate for transition flow (MWH textbook uses 2 as laminar boundary)
            mu = 
            v_c = (((rho_sand - rho_H20)*ac.GRAVITY*diam_p**1.6)/(13.9*rho_H20*nu**0.6))**(1/1.4) 
            Re = v_c*diam_p/nu
            grit_area = q/v_c
      return grit_area

  @property
  def et_influentArea(self):
      """the area required by the influent pipe to reduce kinetic energy"""
      ### Function not defined yet ###
      return et_influentArea
  
  @property
  def lfom_area(self):
      """the area required by the lfom"""
      lfom_area = (self.lfom_nd * 2)**2
      return lfom_area 

  @property
  def et_area(self):
      """the required area of the entrance tank"""  
      et_area = np.max([self.grit_area,self.et_influentArea + self.lfom_area]) #move LFOM_area outside of this equation if we don't want it to count towards the grit chamber
      return et_area

  def etFlocc_specs(self):
      """Calculates the dimensions of the entrance tank, the dimensions of the flocculator, and the number of channels"""
      # Algorithm: Default is an entrance tank in-line with the flocculator. However, if this produces an entrance tank that is longer than a sed tank, then the
      # entrance tank gets its own channel. Additionally, if the first flocculator channel is smaller than one baffle spacing, the entrance tank should also get its
      # own entrance tank for construction purposes. The following is also true, but not necessarily incorporated into the logic: 
              #if et_area/sed_length > flocc_widthMax, then the et MUST get its own channel
              #if et_area/sed_length < flocc_widthMin, then the et MUST be incorporated in the first channel  
      total_area = self.et_area + self.flocc_area
      n_channels = np.ceil(total_area/(self.sed_length * self.flocc_widthMax)) 
      channel_width = total_area/(self.sed_length*n_channels)
      # throw an error if channel_width < channel_min - is this situation physically possible?
      et_length = self.et_area/channel_width
      et_width = channel_width
      flocc_width = channel_width

      if et_length >= self.sed_length: # the entrance tank gets its own channel
         et_length = self.sed_length
         et_width = self.et_area/et_length
         n_chanFloc = np.ceil(self.flocc_area/(self.sed_length * self.flocc_widthMax))
         flocc_width = self.flocc_area/(self.sed_length*n_chanFloc)
         # throw an error if channel_width < channel_min - is this situation physically possible?

      # For contruction purposes, the first channel of the flocculator should have a minimum size, ie. it should be able to fit at least one baffle
      # do I need to create a flocculator with the given length/width to determine the baffle_spacing?
      elif self.sed_length - et_length < self.SF*self.baffle_spacing: # ie. if the first channel of the flocculator is smaller than the baffle spacing, then ET gets its own channel
           et_length = self.sed_length
           et_width = self.et_area/et_length
           if et_width < self.flocc_widthMin:
              et_width = self.flocc_widthMin
           n_chanFloc = np.ceil(self.flocc_area/(self.sed_length * self.flocc_widthMax))
           flocc_width = self.flocc_area/(self.sed_length * n_ChanFloc)
           # update n_channels?  
      return et_width,et_length,flocc_width,flocc_length,n_channels # do I need to separate these into their own definitions/@properties

  @property
  def grit_length(self):
      """the length of the entrance tank containing hoppers on the effluent side of the trash racks"""
      grit_length = 0 * u.m
      if self.grit_area != 0:
         hopper_totalArea = self.grit_area - (self.et_influentArea + self.lfom_area) # adjust this depending on what end up counting as grit area
         grit_length = hopper_totalArea/et_width # does not include the influent hopper. does this need to be self.et_width?
      return grit_length

  @property
  def hopper_width(self):
      """the width of one hopper in the grit chamber"""
      if grit_length != 0:
         hopper_width =  et_width # does this need to be self.et_width?
      return hopper_width

  @property
  def gritHopper_length(self): 
      """Calculates the length of hoppers on the effluent side of the trash rack"""
      gritHopper_length = 0 * u.m
      if self.grit_area != 0:
        gritHopper_length = self.hopper_LWratio * self.hopper_width
        n_hoppers = np.floor(self.grit_length/gritHopper_length)
        # redistribute extra space to each hopper equally
        extraLength = (self.grit_length - n_hoppers*gritHopper_length)/n_hoppers # the extra length added to each hopper to fill the entire grit chamber
        gritHopper_length = gritHopper_length + extraLength
        hopper_LWratioActual = gritHopper_length/self.hopper_width
      return gritHopper_length

  @property
  def influentHopper_length(self): # will there ever be more than one hopper on the influent side?
      """Calculates the length of the hopper on the influent side of the trash rack"""
      influentHopper_length = self.et_influentArea/self.hopper_width
      return influentHopper_length

  def set_drain_pipe(self): # okay to use flocc_depth in this function??
      """Calculates the inner diameter of the entrance tank drain pipes"""
      drain_pipe_k_minor = ac.PIPE_ENTRANCE_K_MINOR + ac.PIPE_EXIT_K_MINOR + ac.EL90_K_MINOR
      nu = ac.viscosity_kinematic_water(self.temp) # should I make viscosity its own @property since it is also used in another function?
      drain_id = ac.diam_pipe(self.q,self.flocc_depth,self.flocc_depth,nu,ac.PVC_PIPE_ROUGH,drain_pipe_k_minor)
      self.drain_pipe = Pipe(id=drain_id,k_minor=drain_pipe_k_minor,spec=self.sdr)

  @property
  def drainfloor_length(self): # assume square geometry
      drainfloor_length = self.drain_pipe.od + 2 * fab_s # is this a valid way to get the outer diameter?
      return drainfloor_length

# Are the following calculations actually helpful for creating the ET in Onshape? I imagine that you would only need the area taken up by the drain and the slope
# Since we already know the overall dimensions of each hopper

  @property
  def gritHopper_lengthWallLong(self): # the distance between the drain floor and the edge of the hopper in the longitudinal direction
      gritHopper_lengthWallLong = 0.5*(self.gritHopper_length - self.drainfloor_length)
      return gritHopper_lengthWallLong

  @property
  def influentHopper_lengthWallLong(self):
      influentHopper_lengthWallLong = 0.5*(self.influentHopper_length - self.drainfloor_length)
      return influentHopper_lengthWallLong  
    
  @property
  def hopper_lengthWallTrans(self): # the distance between the drain floor and the edge of the hopper in the transverse direction
      hopper_lengthWallTrans = 0.5*(self.hopper_width - self.drainfloor_length)
      return hopper_lengthWallTrans
  
  @property
  def gritHopper_depthLong(self): # the distance between the bottom of the ET and the top of the hopper in the longitudinal direction
      gritHopper_depthLong = self.hopper_slope * self.gritHopper_lengthWallLong
      return gritHopper_depthLong
  
  @property
  def influentHopper_depthLong(self):
      influentHopper_depthLong = self.hopper_slope * self.influentHopper_lengthWallLong
      return influentHopper_depthLong

  @property
  def hopper_depthTrans(self): # the distance between the bottom of the ET and the top of the hopper in the transverse direction
      hopper_widthTrans = self.hopper_slope * hopper_lengthWallTrans


In [ ]:
#### Miscellaneous (Old) Code ####

# scrap code -  just testing out some numbers
a = 5
lfom_od = 0.3
L_sed = 7.1
n = 4
b = 3 * lfom_od - n*L_sed
w_flocculator = 0.60 * u.m # the width of one flocculator at Gracias (inner width ie. does not include thickness of the concrete)
length_flocculator = 7 * u.m # the length of one flocculator channel at Gracias (inner length ie. does not include thickness of the concrete)
A_flocc = 3*w_flocculator*length_flocculator + w_flocculator*4.98*u.m
print(b)
print(A_flocc)

def grit_chamber(q,T,d_p): # **complete**
    """Calculates the required area of a grit chamber given plant flow rate, temperature, and critical particle size"""
    rho_H20 = ac.density_water(T)
    rho_p = 2650 *u.kg/u.m**3 # density of sand
    nu = ac.viscosity_kinematic_water(T) # kinematic viscosity
    v_c = ((rho_p - rho_H20)*ac.GRAVITY*d_p**2)/(18*nu*rho_H20)
    Re = v_c*d_p/nu
    if Re > 2: # recalculate for transition flow (MWH textbook uses 2 as laminar boundary)
       v_c = (((rho_p - rho_H20)*ac.GRAVITY*d_p**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4) 
       Re = v_c*d_p/nu
    grit_area = q/v_c
    return grit_area

d_p = (0.35 * u.mm).to(u.m)
c = grit_chamber(q,T,d_p)
print(c)

# Design the entrance tank in python (note: will probably rewrite this in a class)

def grit_chamber(q,T,d_p): # **complete**
    """Calculates the required area of a grit chamber given plant flow rate, temperature, and critical particle size"""
    rho_H20 = ac.density_water(T)
    rho_p = 2650 *u.kg/u.m**3 # density of sand
    nu = ac.viscosity_kinematic_water(T) # kinematic viscosity
    v_c = ((rho_p - rho_H20)*ac.GRAVITY*d_p**2)/(18*nu*rho_H20)
    Re = v_c*d_p/nu
    if Re > 2: # recalculate for transition flow (MWH textbook uses 2 as laminar boundary)
       v_c = (((rho_p - rho_H20)*ac.GRAVITY*d_p**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4) 
       Re = v_c*d_p/nu
    grit_area = q/v_c
    return grit_area

def et_area(q,T,d_p):
    """Calcualtes the required area of the entrance tank given plant flow rate, temperature, and desired particle size to remove"""
    grit_area = grit_chamber(q,T,d_p)
    LFOM_area = (lfom_nd * 2)**2
    # influent_area = (includes tRacks)
    et_area = np.max([grit_area,influent_area + LFOM_area]) #move LFOM_area outside of this equation if we don't want it to count towards the grit chamber
    return et_area

# Algorithm: Default is an entrance tank in-line with the flocculator. However, if this produces an entrance tank that is longer than a sed tank, then the
# entrance tank gets its own channel. Additionally, if the first flocculator channel is smaller than one baffle spacing, the entrance tank should also get its
# for construction purposes. The following is also true, but not necessarily incorporated into the logic: 
      #if et_area/sed_length > flocc_widthMax, then the et MUST get its own channel
      #if et_area_sed_length < flocc_widthMin, the the et MUST be incorporated in the first channel

def etFlocc_specs(flocc_area,flocc_widthMax,flocc_widthMin,sed_length,q,T,d_p):
    """Calculates the dimensions of the entrance tank, the dimensions of the flocculator, and the number of channels"""
    et_area = et_area(q,T,d_p)
    total_area = et_area + flocc_area
    n_channels = np.ceil(total_area/(sed_length * flocc_widthMax)) 
    channel_width = total_area/(sed_length*n_channels)
    # throw an error if channel_width < channel_min - is this situation physically possible?
    et_length = et_area/channel_width
    et_width = channel_width
    flocc_width = channel_width

    if et_length >= sed_length: # the entrance tank gets its own channel
       et_length = sed_length
       et_width = et_area/et_length
       n_chanFloc = np.ceil(flocc_area/(sed_length * flocc_widthMax))
       flocc_width = flocc_area/(sed_length*n_chanFloc)
       # throw an error if channel_width < channel_min - is this situation physically possible?

    # For contruction purposes, the first channel of the flocculator should have a minimum size, ie. it should be able to fit at least one baffle
    elif sed_length - et_length < SF*baffle_spacing: # ie. if the first channel of the flocculator is smaller than the baffle spacing, then ET gets its own channel
       et_length = sed_length
       et_width = et_area/et_length
       if et_width < flocc_widthMin:
          et_width = flocc_widthMin
       n_chanFloc = np.ceil(flocc_area/(sed_length * flocc_widthMax))
       flocc_width = flocc_area/(sed_length * n_ChanFloc)
       # update n_channels?  
    return et_width,et_length,flocc_width,flocc_length,n_channels

hopper_area = grit_area - (influent_area + LFOM_area)

### Hopper Design ###

def hopper_design(width_ET,length_ET):
    """Determine the number of hoppers and their dimensions"""
    LW_ratio_design = 1 # dimensionless variable used to manipulate the tradeoff between hopper depth and number of hoppers
    width_hopper = width_ET
    length_hopper = LW_ratio_design*width_hopper
    n_hoppers = np.floor(length_ET/length_hopper)
    # redistribute extra space to each hopper equally
    extraLength = (length_ET - n_hoppers*length_hopper)/n_hoppers # the extra length added to each hopper to fill the entire grit chamber
    length_hopper = length_hopper + extraLength
    LW_ratio_actual = length_hopper/width_hopper

    # the minimum number of hoppers is 2 (either side of the trash racks) need to flesh out this scenario

    print('There are ',n_hoppers.magnitude, 'hoppers with a length:width ratio of ', LW_ratio_actual)
    print('The width of each hopper is ', width_hopper)
    print('The length of each hopper is ',length_hopper)

    # determine the diameter of the drain pipe
    t_drain = (5 * u.min).to(u.s) # this was an arbitrary choice
    L_tank = length_ET # need to think more about this
    W_tank = width_ET # need to think more about this
    H_tank = depth_flocculator # for now. we haven't decided how deep it will actually be
    sum_K = ac.PIPE_ENTRANCE_K_MINOR + ac.PIPE_EXIT_K_MINOR + ac.EL90_K_MINOR
    ID_drainpipe = ((8*L_tank*W_tank)/(np.pi*t_drain))**0.5*((H_tank*sum_K)/(2*ac.GRAVITY))**0.25
    print('The inner diameter of the hopper drainpipe is ',ID_drainpipe)
    # looking at old ET code in aguaclara/design/ent.py, it uses a different method to find the 
    # drain pipe size which is through ac.diam_pipe(q,headloss,length,nu,piperough,kminor)

    # need to ensure that the inner diameter is actually one that exists for common pipes
    #OD_drainpipe = (ac.OD).to(u.m)

    # determine the hopper depth
    fab_s = (5 * u.cm).to(u.m) # the space needed for a person to remove the drain pipe (reference aguaclara/design/ent.py)
    s_drainfloor = ID_drainpipe + 2*fab_s # the drain floor is a square with the drain pipe in the middle. This is the side-length of that square.
    # s_drainfloor needs to be calculated with OD_drainpipe, but I used ID_drainpipe for now so the code would run

    # the angle of repose for wet sand is 45 degrees ie. slope = 1
    slope_zx = 1 # minimum allowable slope for the hopper wall in the longitudinal direction 
    slope_zy = 1 # minimum allowable slope for the hopper wall in the transverse direction
    length_hopperWall_long = 0.5*(length_hopper - s_drainfloor)
    length_hopperWall_trans = 0.5*(width_hopper - s_drainfloor)
    depth_hopper_long = slope_zx * length_hopperWall_long # the distance between the bottom of the ET and the top of the hopper in the longitudinal direction
    depth_hopper_trans = slope_zy * length_hopperWall_trans # the distance between the bottom of the ET and the top of the hopper in the transverse direction

    print('The depth of the hopper in the longitudinal direction is ', depth_hopper_long.to(u.cm))
    print('The depth of the hopper in the transverse direction is ', depth_hopper_trans.to(u.cm))

def et_length(floc_width):
        # Important Et Parameters that define it's length
    length_LFOM = 2*nd_LFOM 
    length_LFOMspace = (80 * u.cm).to(u.m) # the space needed at the end of the ET, if it is the full length of the flocculator
    length_gritChamber # depends on floc width
    length_tRacks # longitudinal space that the grates take up.
    length_influent # depends on ET width
    et_length = LFOM_length + LFOMspace_length + grit_length + tRacks_length + influent_length
    return et_length

# calculate dimensions of the entrance tank (assuming it includes all essential ET components like the LFOM, etc.)
d_p = (0.25 * u.mm).to(u.m) # minimum grit diameter to be removed (arbitrary number chosen - depends on water source)
v_c = ((rho_p - rho_H20)*ac.GRAVITY*d_p**2)/(18*nu*rho_H20)
Re = v_c*d_p/nu
if Re > 2: # recalculate for transition flow (MWH textbook uses 2 as laminar boundary)
   v_c = (((rho_p - rho_H20)*ac.GRAVITY*d_p**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4) 
   Re = v_c*d_p/nu
area_ET = q/v_c
print('The area of the entrance tank is',area_ET)
width_ET = w_flocculator
length_ET = ((area_ET/width_ET).magnitude) * u.m # Need to think about how the area taken up by the LFOM, trashracks, and influent pipe might fit into this
if length_ET > length_flocculator:
   length_ET = length_flocculator
   width_ET = area_ET/length_ET
#length_min = length_LFOM + length_tRacks + OD_influent pipe #Figure out how much space the components inside the ET take up 
#if length_ET < length_min:
#   length_ET = length_min

print('The length of the entrance tank is ',length_ET)
print('The width of the entrance tank is ', width_ET)

# trash racks 
# made of #10 SS round-wire mesh 2 mm diameter
# at Gracias, the trash rack is a pvc sheet with 1.3 cm holes drilled into it; however trash racks can also be made with SS mesh

#n_tRack = 2 # number of trash racks
#n_tRackSlots = n_tRack + 1 # always need an empty slot so that the order of trash racks can be rearranged 
#s_tRack = (8 * u.cm).to(u.m) # center-center distance between trash racks (8 cm is an arbitrary guess)
#n_orifices = # number of orifices per trash rack
#d_orifices = # orifice diameter
#thickness_tRack = # depends on material probably
#width_tRack = width_ET
#height_tRack = depth_flocculator

# influent pipe

# potential diffuser wall

# add safety factors?

-27.5
15.59 meter ** 2
1.285 meter ** 2


NameError: ignored

# Information and Notes

## Executive Summaries

### Executive Summary: June 22 - July 12 2020 

**Entrance Tank Size**

The trash rack and plate settlers in the entrance tank at Gracias have been unsuccessful at capturing grit and preventing it from traveling to the rest of the plant. This grit causes maintenance issues by settling in the flocculator and the channel leading to the sedimentation tanks. To solve this problem, the plate settlers could be replaced with a more traditional horizontal settling basin.

The required plan-view area of a horizontal settling basin is determined by the terminal velocity of the smallest particle it is designed to remove. Kawamura suggests that sand and silt smaller than 0.1 mm do not pose a significat risk to downstream pumps and processes (pg. 416). **However, to settle particles larger than 0.1 mm in diameter, the entrance tank would need to be 11.13 m in length.** This calculation was done under the assumption of a 60 L/s flow rate, and that the entrance tank is the same width as the flocculator. For comparison, a single treatment train at the Gracias plant (excluding the filters) covers an approximate 15 m x 7 m area. Increasing the area of the entrance tank would drive up construction costs and may block plant operator access to important parts of the plant. 

The required plan-view area of the entrance tank could be reduced by increasing the critical particle size that it is designed to capture. Any particles that are not captured in the entrance tank must be scoured in the flocculator and the inlet channel of the sedimentation tanks, otherwise they will cause maintenance issues. Calculations should focus on the inlet channel of the sedimentation tanks since it has lower velocities and less shear than the flocculator. 

Two approaches were taken to calculate the smallest particle that needs to be captured by the entrance tank. The first approach approximates the scouring velocity in the inlet channel with the following equation: $$V_{min} = \sqrt{\frac{2}{3}\frac{\sqrt{Re_x}}{0.332}\frac{\rho_{p} - \rho_{H20}}{\rho_{H20}}\ g\ d_p\ tan\theta}$$

The second approach approximates the shear on the bottom of the inlet channel and uses a modified Shields diagram to determine the corresponding critical particle diameter. **Both approaches produced a critical particle diameter (2.373 mm and 7.5 mm respectively) that is larger than the typical size of sand. These results are not physically reasonable, and do not match observations made by plant operators in Gracias.** The key assumptions made in each approach should be re-examined or a new model should be developed. 

After consulting Agua Para el Pueblo engineers, it was determined that sand carryover is not as big of a problem in AguaClara plants as originally thought. Gracias and Agalteca are the only two plants that are having this issue. Therefore it makes sense that the entrance tank should be designed so that a grit chamber can be easily incorporated for water sources with potential sand carryover issues. Depending on the flow rate and the need for grit removal, the entrance tank design will range from 1) an entrance tank in-line with the flocculator and with the same width as the flocculator, to 2) an entrance tank the entire length of the flocculator channel and the same width as the floccutor, and 3) to an even wider entrance tank.

**Entrance Tank Kinetic Energy**

Wave action and splashing is another issue occuring in the entrance tank at the plant in Gracias, Honduras. The velocity and kinetic energy was calculated at various points within the entrance tank (Table 1): exiting the influent pipe, through the plate settlers, and at the shallow channel leading to the LFOM. 

*Table 1: The velocity and kinetic energy at various location in the entrance tank*

|Location|Velocity (m/s)|Kinetic Energy (cm)|
|:---:|:---:|:---:|
|Influent Pipe|1.91|18.6|
|Plate Settlers|0.16|0.1305|
|LFOM Channel|0.1593|0.1294|

The velocity exiting the influent pipe is much greater than other areas of the entrance tank. **This suggests that the kinetic energy exiting the influent pipe may be the primary source of wave action.** Various methods should be explored to reduce this kinetic energy. This design may include the following components:
- aiming the influent pipe at the end wall of the entrance tank
- use 2" pipes as diffusers
- placing obstacles along the side walls of entrance tank
- including a diffuser wall (the trash racks may assist with this)
- place an endcap on the influent pipe so that water does not flow straight down into the hopper, and cut a vertical slit down the side to act as a diffuser. 

Each component should be evaluated on its effectiveness at dissipating kinetic energy and its ease of construction.

**Next Steps:**
- Determine if the addition of a second trash rack will help diffuse kinetic energy
- Determine if a diffuser wall or baffles/obstacles are better at diffusing kinetic energy, and which one is more convenient to construct. 






### Executive Summary: June 22 - June 28 2020  
The trash rack and plate settlers in the entrance tank at Gracias have been unsuccessful at capturing grit and preventing it from traveling to the rest of the plant. This grit causes maintenance issues by settling in the flocculator and the channel leading to the sedimentation tanks. To solve this problem, the plate settlers could be replaced with a more traditional horizontal settling basin. A series of 4-sided hoppers fitted with a drainpipe would make it easy to remove grit from the bottom of the chamber via gravity. However, Kawamura advises against the use of square tanks, due to their poor flow patterns:

> “The square tank is satisfactory for the purpose of wastewater treatment but does not perform well as a water treatment grit chamber. The main problem with the detritus [square] tank is a function of its physical characteristics. The short tank length and the uneven inflow to the tank cause severe flow short-circuiting. … To provide good flow characteristics for grit settling, the chamber should be rectangular with a contracted inlet (pg. 414).” 

**To settle particles greater than 0.1 mm in diameter, the entrance tank at Gracias would need to be 11.13 m in length.** This calculation was done under the assumption of a 60 L/s flow rate, and that the entrance tank is the same width as the flocculator.  According to Kawamura, sand and silt smaller than 0.1 mm do not pose a significant risk to downstream processes (pg. 416). For context, a single treatment train (excluding the filters) at Gracias has an approximate 15 m x 7 m area. Increasing the area of the entrance tank would drive up construction costs and may block plant operator access to important parts of the plant. For example, observing the flocs in the first 4 baffles of the flocculator is one of the most reliable indications of plant performance. 

The required plan-view area of the entrance tank could be reduced by increasing the critical particle size that needs to be captured. Kawamura recommended a 0.1 mm diameter based on pump and piping damage in downstream processes. However, the predominant issue in Gracias, Honduras is grit settling in the flocculator and inlet channel to the sedimentation tanks.

The velocities going around the bottom bend of the flocculator are comparatively higher than velocities in the entrance tank. This suggests that particles larger than 0.1 mm could be scoured from the bottom of the flocculator, and that the entrance tank may be designed for a larger critical particle diameter. **The shear on the bottom of the flocculator should be calculated to determine the largest particle that can be scoured.**

Wave action and splashing is another issue occuring in the entrance tank at the plant in Gracias, Honduras. The velocity and kinetic energy was calculated at various points within the entrance tank (Table 1): exiting the influent pipe, through the plate settlers, and at the shallow channel leading to the LFOM. 

*Table 1: The velocity and kinetic energy at various location in the entrance tank*

|Location|Velocity (m/s)|Kinetic Energy (cm)|
|:---:|:---:|:---:|
|Influent Pipe|1.91|18.6|
|Plate Settlers|0.16|0.1305|
|LFOM Channel|0.1593|0.1294|

The velocity exiting the influent pipe is much greater than other areas of the entrance tank. **This suggests that the kinetic energy exiting the influent pipe may be the primary source of wave action.** Various methods should be explored to reduce this kinetic energy. This design may include the following components:
- aiming the influent pipe at the end wall of the entrance tank
- use 2" pipes as diffusers
- placing obstacles along the side walls of entrance tank
- including a diffuser wall (the trash racks may assist with this)
- place an endcap on the influent pipe so that water does not flow straight down into the hopper, and cut a vertical slit down the side to act as a diffuser. 

**Each component should be evaluated on its effectiveness at dissipating kinetic energy and its ease of construction.** 


## Information sources
1.  [Capstone Design: Improved Entrance Tank Design](https://github.com/AguaClara/CEE4520/blob/master/Previous_Final_Projects/2019F/Improved_Entrance_Tank.ipynb)
1.   MWH textbook
1.   [Susumu Kawamura textbook](https://photos.app.goo.gl/6KWgu6m9uSKmRAKH9)
1.   [Sed tank design code example](https://colab.research.google.com/github/AguaClara/CEE4520/blob/master/Unit_Process_Designs/Sed.ipynb)
1. [Gracias CAD Design](https://cad.onshape.com/documents/cb0559d6c3bd85907f8704d3/w/8b31f12d476b14cf91bd29d9/e/6d7584b2804b3c166ba1e991)
1. [Hydraulic Engineering: Channel Design Powerpoint](https://github.com/monroews/Hydraulics/raw/master/10_Channel_Design.pptx) (slides 19-23)
1. Chapter 12: Stable Channel Design Functions in [HEC-RAS River Analysis System Hydraulic Reference Manual](https://www.hec.usace.army.mil/software/hec-ras/documentation/HEC-RAS%205.0%20Reference%20Manual.pdf)  (page 12-21)

## Problems with the current design

1.   Explore options to reduce the kinetic energy of the incoming water. This results in lots of wave action and splashing. Perhaps inject the water deeper into the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface. Or direct the water against the wall of the tank? Perhaps  2" diameter diffusers (so that clogging isn't a problem) in the vertical entrance pipe with the diffusers pointing upstream toward the end wall of the entrance tank? 
1.   Plate settlers don't work because the velocities are too high and flow isn't distributed at all unformly through the plates. Recommend either improving flow distribution (see 4520 capstone design project) or reverting to a simple horizontal flow tank to capture grit. The simple horizontal sed tank should be explored as the simplest option.
1.   Velocity through the bar screen is too high. Increase the area for the bar screen and include a minimum of 2 bar screens and 3 tracks for inserting the screens so that one can be removed and cleaned and replaced. Screen can be angled to enable it to have more surface area.
1.   Velocity of approach for the LFOM is too high causing wave action. See if we can predict the wave action that is observed at Gracias. The waves are simple a conversion of kinetic energy into potential energy, $v=\sqrt{2gh}$. 

The new entrance tank will likely take more horizontal space OR it is likely that it simply is the full depth of the flocculator with several hoppers that drain any accumulated sediment.

## Strategy

1.  Design the ET in python
1.  Draw the ET in Onshape
1.  Transition on the python code to Feature Script
1.  Intergrate the new ET with the flocculator, LFOM, and CDC



## Notes from meeting with Monroe








### Mon. July 13

- Floc Hopper
  - there is no need to have a flat space in between hoppers except where the trash racks are located
  - when there is extra space at the end of the ET, round up instead of down?
  - the angle of the floor given by Kawamura (1:100) is only helpful for getting the last few millimeters of water out of the tank. Therefore we should use and angle closer to 60 degrees. The angle of repose for sand is 45 degrees when wet. The angle for clay is typically lower, so choose 45 degrees as the conservative angle. 
  - the depth of the floc hopper is controlled by the longer dimension (length of width). There is a tradeoff between the the depth of the hopper and the number of hoppers. Use a dimensionless number so that these parameters can easily be adjusted (it's easier to justify a design this way)
- Kinetic Energy
- Outer ET dimensions
- LFOM



---

5 L/s plant minimium case with no waves, it will have a narrower
flocculator channe, minimum 45 cm wide (human width)



we know how deep the flocc

sketch out minimum and makimum cases 

is there a maximum width of the flocculator channels
write equations to calculate the actual width and lenght of the ET



we know the plan view area of the flocculator for sure
flocculator does not  know how wide it is until
it knows how wide the ET is
Length occupied by LFOM is 3 times outer diameter of the LFOM


case minimum channel * length sed
anytime the et pv area/minimum width of a channel (45 cm)
is greater than length of a sed channel the ET gets it own channel

LFOm leading to the flocculator
make it an L (in either direction)
if et full elngth of channel there is a space at the end
at least 80 cm for a human to fit there will be the most
turbulence there

ET includes 80 cm of open space after the LFOM and this will
already show up for big plants

we can decrease velocities by having the coupling that the
LFOM plugs into lower the ledge significantly 
the holes are at the very top
where does the chemical get inejcted
we would like to inject the coagulantbelow the free fall
of the LFOM, maybe below the level of water that  the flocculator
will back up to 

add 10 cm free fall in the flocculator

inject the chemical where there's always water
downstream water level of the flocculator
there's a half inch diam pipe going into the LFOM pipe
the coagulant tubing should go through the wall of the
entrance tank








### Wed. June 24
- clarification: we are concerned with splashing everywhere in the ET
- start with worst case scenario, if the pipe coming into the entrance tank was directed right towards the surface. Take the velocity coming out of influent pipe, and find the velocity flowing uniformly between the plates. There is very little headloss between the plates, so it's difficult to get an even distribution of flow
- From powerpoint about turbulence:
  - b is the width of the jet
  - Vc is the velocity at the center of the jet
  - s equals the distance forward
  - there is a pproximately a 1/10 ratio between s and b, ie. b = 0.116s or something like that (number might not be exactly correct). Suppose the orifices on a trash rack are spaced out a centimeter. This means that 10 cm out, all of the jets will have merged.
- We want to create many small eddies, because they dissipate the fastest, the trash rack can possibly help with this.
  - Make the trash rack taller and potentially at a slope to increase surface area.
  - at the far end of the flocculator, the channel is 2 m deep (we think), upstream add 50 cm headloss, then add 10 cm drop in the LFOM, and then an additional 20 cm for a total of 2.8 m height in the entrance tank. The trash racks would be placed after the first hopper and would go as deep as the top of the hopper. There is a physical limitation to how deep the trash rack can be, because the operators have to be able to lift it to clean it.
- There are going to be 4 sided hoppers, need to determine how many and the length of the entrance tank. The number of drains depends on the size of particles. Confirm capture velocity from Kawamura textbook being 8 mm/s and this will set the length of the sed tank
- the influent pipe would have a firnco coupling so it can be taken out, and possibly capped at the bottom end to prevent water from flowing straight down into the first hopper. This would be bad because it would not promote even flow throughout the tank. Influent pipe could also have a slot on the side instead of diffusers.
- the eddy velocity in a pipe is roughly 10% of the mean velocity

**Action Items:**
- Monroe will send the pictures of the Kawamura textbook **complete**
- Calculate v^2/2g for the plate settlers as Gracias. 5 cm would be a lot. **complete**
- Calculate the velocity in the channel leading to the LFOM (where it gets a lot more shallow really fast) **complete**
- Calculate how long the ET would need to be for a 60 L/s plant when designed as a horizontal sedimentation chamber. Assume the same flocculator width as Gracias. **complete**

**Reflections & Future Questions**
- Will flow expansion around a barrier dissipate more kinetic energy than water flowing through a barrier with a bunch of hole drilled into it? 
- Do non-AguaClara entrance tanks have similar problems? Can we learn anything from how problems have been dealt with in the past? 
- is the energy dissipation rate of water coming out of the influent pipe related to the energy dissipation rate of the turbulent eddies?
- For predicting wave action around LFOM, what observations at Gracias should this be based off of?
- are there any drawbacks to injecting the water deeper in the entrance tank?
- the bottom geometry of the entrance tank should be selected to ensure that maintenance and the removal of grit is simple and effective. 

**Kawamura Textbook Notes:**

- "a hydraulic scale model conducted by the author clearly demonstrated that when the screen is installed properly, it functions as an inlet diffuser wall"
- "The square tank is satisfactory for the purpose of wastewater treatment but does not perform well as a water treatment grit chamber. The main problem with the detritus [square] tank is a function of its physical characteristics. The short tank length and the uneven inflow to the tank cause severe flow short-circuiting. ... To provide good flow characteristics for grit settling, the chamber should be rectangular with a contracted inlet."

**MWH Textbook Notes:**
"Although coarse screens are used with river intakes, a fine debris screen
with approximately 20-mm openings is often provided at the front end
of the presedimentation tank. Because the screen also acts as an effective
diffuser wall, it should be installed close to the tank inlet. If a separate
diffuser wall is specified, the total area of openings at the wall should be
about 15 percent of the tank cross-sectional area" (pg. 667)

**Questions:**


- how much bigger does the bottom floor drain need to be than the drainpipe? **example in /aguaclara/designs/ent.py**

- should the area of essential ET components (ie. LFOM, trash rack, influent pipe) count towards area needed for settling particles?

**Trash Racks**

- Considerations for determining the size of holes
  - what size particles/debris are we trying the remove?
  - in Rosie's design, the trash racks are protecting the plant from particles larger than 2 mm sand
  - do not want a super high velocity through the holes (put as many as possible over a large area)
- plant operators need to be able to lift the trash racks
  - really tall trash racks, large velocities/high KE might make it for operators to lift
- should the trash racks be at an angle to increase surface area? Need to balance the need to reduce velocities and making the rack physically easy to handle
- how close should the racks be spaced apart?
- what is the point of layering the wire mesh with the rebar? Why can't you rake the trash rack without the rebar?
- Is it necessary for both trash racks to be in the tank at the same time? 
  - Pros: having both trash racks in the tank give a second chance to capture particles...but if the mesh is the same size, shouldn't they capture the same size particles. I don't see how the second screen would help that much.
  - another reduction of kinetic energy
- random idea: what if the trash rack had a hinge on it so you could fold it in half and make it easier to carry? This might not be ideal since it creates a moving part that could break and would need to be replaced
- why were the holes at Gracias made so big (1.3 cm)?
- what is the relationship between headloss and the creation of turbulent eddies? Does the source of headloss control the size of turbulent eddies produced? 




## Task: Reduce the kinetic energy of the influent water

**Problem:** The high kinetic energy of the influent water causes lots of wave action and splashing 

**Possible Solutions**
1. inject the water deeper into the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface.
1. direct the water against the wall of the tank
1. use 2" diameter diffusers (so that clogging isn't a problem) in the vertical entrance pipe with the diffusers pointing upstream toward the end wall of the entrance tank



### Task 1 Solution 1: Inject water deeper in the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface

**Edit 6/24: not sure that we really need to any math to determine how deep the influent tube should be. would probably just position it as low as possible. Ask Monroe later.**

- Define "somewhat dissipated."
  - From AC textbook: "Eddy turnover time, $t_{eddy}$, is the time it takes for the eddy to travel a distance equal to its length-scale. Thus the eddy turnover time provides a good estimate of the time required for mixing to occur at the length scale of the eddy. **We assume that the energy of the large eddy is dissipated into smaller length scales in the time $t_{eddy}$:**"

- Which equations govern the dissipation of turbulent eddies?
$$t_{eddy} \approx \frac{L_{eddy}}{v_{eddy}}$$

   The rate of energy loss to smaller scales
$$\bar\varepsilon \approx\frac{v_{eddy}^3}{L_{eddy}}$$

   and therefore...

$$v_{eddy} \approx \left( \bar\varepsilon \, L_{eddy} \right)^\frac{1}{3}$$

- "The largest eddies are limited in size by the smallest dimension normal to the direction of flow"

   *Therefore the length scale of a turbulent eddy in the entrance tank is either the vertical distance between the water surface and the 'baffle wall', or the width of the entrance tank (whichever distance is smaller).*???

- How is water currently being injected?
  - see onshape drawing, there's a tee at the end of the pipe so water flows out horizontally towards the side walls of the entrance tank

- For reference: The form of energy loss from the influent pipe is expansion, (ie. the following equations are relevant)

$$h_e = K\frac{\bar v_{out}^2}{2g}$$

$$\bar\varepsilon = K\frac{\bar v_{out}^3}{2H}$$

$$G_{CS} = \bar v_{out}\sqrt{\frac{K\bar v_{out}}{2H\nu}}$$

 
- Determine the injection depth needed so that turbulent eddies are "somewhat dissipated"
  
